In [1]:
######
##############################Execution##########################
##############################
###### Load The Delta   ######
##############################
###
### Input delta in folder :  /data 
my_input_delta_table="delta_real_estate_term_definitions"
###
######
##############################Execution##########################
import findspark
findspark.init()
#
import pyspark
from pyspark.sql import functions as pfunc
from pyspark.sql import SQLContext
from pyspark.sql import Window, types
import re
import pandas as pd
import numpy as np
from pandas import DataFrame
from pyspark.sql.types import IntegerType
from pyspark.sql.types import FloatType
from pyspark.sql.functions import udf
from pyspark.sql.functions import *
from scipy.stats import kstest
from scipy import stats
#
import subprocess
#
sc = pyspark.SparkContext(appName="Business_Dictionary-Delta")
sqlContext = SQLContext(sc)
#
#
internal_delta_files="file:///home/notebookuser/notebooks/data/"+my_input_delta_table
#
# Join with Internal Curation Data in urltopredict staged folder
from pyspark.sql import functions as F
### use version=1
version=1
## .option("versionAsOf", version)
delta_dataframe_df1=sqlContext.read.format("delta").load(internal_delta_files)\
.persist(pyspark.StorageLevel.MEMORY_AND_DISK_2)
#
##.read.format("delta").load("/delta/events")
#
delta_dataframe_df1.printSchema()
delta_dataframe_df1.registerTempTable("real_estate_terms")
#
#
#######################################################
#########  Evaluate Frequency of Real Estate Terms  ###
#######################################################
##
nfolder="delta_terms_words_ngrams_real_estate"
out_file1="file:///home/notebookuser/notebooks/data/"+nfolder
##
from pyspark.ml.feature import Tokenizer
from pyspark.ml.feature import RegexTokenizer
#
#import org.apache.spark.ml.feature.NGram
from pyspark.ml.feature import NGram
#
from collections import Counter
#
from pyspark.ml.feature import NGram
#
from pyspark.ml.feature import NGram, CountVectorizer, VectorAssembler
from pyspark.ml import Pipeline
#
from pyspark.mllib.linalg import SparseVector, DenseVector
#
from pyspark.ml.feature import PCA
from pyspark.ml.linalg import Vectors
#
# Join with Internal Curation Data in urltopredict staged folder
from pyspark.sql import functions as F
#
regexTokenizer = RegexTokenizer(minTokenLength=1, gaps=False, pattern='\\w+|', inputCol="Real_Estate_Definition", outputCol="words", toLowercase=True)
#
delta_dataframe_df4=sqlContext\
.sql("Select Real_Estate_Term, Real_Estate_Definition from real_estate_terms where Real_Estate_Term IS NOT NULL")\
.withColumn('hash_real_estate_term',F.sha2(col('Real_Estate_Term'),256))\
.persist(pyspark.StorageLevel.MEMORY_AND_DISK_2)
#
tokenized_DF = regexTokenizer.transform(delta_dataframe_df4)
#
ngram = NGram(n=4, inputCol="words", outputCol="ngrams_4")
countvector = CountVectorizer(inputCol="ngrams_4", outputCol="ngramscounts_4")
# fit a CountVectorizerModel from the corpus.
countvModel = CountVectorizer(inputCol="words", outputCol="features_15", vocabSize=15, minDF=2.0)
# fit a PCA Dimensionality reduction into 7/3=2.x components from ngramscounts_4 ## Too Heavy 1st PCA
pcaNgrams = PCA(k=3, inputCol="ngramscounts_4", outputCol="pcaweightngrams")
# fit a PCA Dimensionality reduction into 85/17=5 components from words
pcaWords = PCA(k=3, inputCol="features_15", outputCol="pcaweightwords")  ## Too Heavy 2nd PCA
#
ngram_dtr_DF = ngram.transform(tokenized_DF)
#
ngram_vc_DF = countvector.fit(ngram_dtr_DF).transform(ngram_dtr_DF)\
.persist(pyspark.StorageLevel.MEMORY_AND_DISK_2)
ngram_vc_DF.printSchema()
#
result_ngrams_words_DF = countvModel.fit(ngram_vc_DF).transform(ngram_vc_DF)\
.persist(pyspark.StorageLevel.MEMORY_AND_DISK_2)
result_ngrams_words_DF.printSchema()
#
#modelPCA_features_ngram_fraud_DF = pcaWords.fit(result_ngrams_words_fraud_DF).transform(result_ngrams_words_fraud_DF)\
#.persist(pyspark.StorageLevel.MEMORY_AND_DISK_2)
#modelPCA_features_ngram_fraud_DF.printSchema()
#
result_ngrams_words_DF.coalesce(1).write.format("delta").save(out_file1)
#
print("Calculation of most frequent Real Estate terms as Ngram4 - Finished!")

root
 |-- Real_Estate_Term: string (nullable = true)
 |-- Real_Estate_Definition: string (nullable = true)

root
 |-- Real_Estate_Term: string (nullable = true)
 |-- Real_Estate_Definition: string (nullable = true)
 |-- hash_real_estate_term: string (nullable = true)
 |-- words: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- ngrams_4: array (nullable = true)
 |    |-- element: string (containsNull = false)
 |-- ngramscounts_4: vector (nullable = true)

root
 |-- Real_Estate_Term: string (nullable = true)
 |-- Real_Estate_Definition: string (nullable = true)
 |-- hash_real_estate_term: string (nullable = true)
 |-- words: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- ngrams_4: array (nullable = true)
 |    |-- element: string (containsNull = false)
 |-- ngramscounts_4: vector (nullable = true)
 |-- features_15: vector (nullable = true)

Calculation of most frequent Real Estate terms as Ngram4 - Finished!
